Imports

In [44]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
from pandas_scd import scd2

Connection to db file

In [3]:
connnection = sqlite3.connect("sqlite.db")
cursor = connnection.cursor()

Table Geografia initialization - SCD Type 2 by Region

In [94]:
#Manualy changed few regions, than read csv file
df_change = pd.read_csv("economist_country_codes.csv", sep=";")
df_change.rename(columns={'COUNTRY|CODE': 'Kraj', 'REGION' : 'Region'}, inplace=True)
#Getting data from DB of still active records
df_DB = pd.read_sql("select * from Geografia where End_Data is Null", connnection)
df_DB['is_active'] = True
df_DB.rename(columns={'COUNTRY|CODE': 'Kraj', 'REGION' : 'Region', 'Start_Data': 'start_ts', 'End_Data': 'end_ts'}, inplace=True)

In [95]:
date_to_raw = datetime.datetime.now().date()

In [96]:
df_target = scd2( df_change, df_DB)

+------------------------------------------+-------------+
| category                                 |   row count |
|------------------------------------------+-------------|
| unchanged active keys (active - old)     |          56 |
| unchanged inactive keys (inactive - old) |           0 |
| ended keys (inactive - new)              |           2 |
| new keys (active - new)                  |           2 |
| total                                    |          60 |
+------------------------------------------+-------------+


In [98]:
df_target = df_target.drop(df_target.columns[-1], axis=1)

In [116]:
df_target['start_ts'] = pd.to_datetime(df_target['start_ts'], format='mixed').dt.date
df_target['end_ts'] = pd.to_datetime(df_target['end_ts'], format='mixed').dt.date
df_target.rename(columns={'start_ts': 'Start_Data', 'end_ts': 'End_Data'}, inplace=True)

In [150]:
update_temp = pd.concat([df_change, df_DB]).drop_duplicates(subset=['Kraj', 'Region'], keep=False)
update_temp = update_temp.drop_duplicates(subset=['Kraj'], keep='first')

In [196]:
update_db = pd.DataFrame()
for kraj_name in update_temp['Kraj']:
    update_db = pd.concat([update_db, df_target[df_target['Kraj'] == kraj_name]])
update_db = update_db.replace([pd.NaT], [None])

In [199]:
sql_insert_update_query = "insert into Geografia (ID_Kraj, Kraj,Region, Start_Data, End_Data) values(?,?,?,?,?) ON conflict(ID_Kraj) do update set End_Data = End_Data;"
for i in update_db.to_numpy():
    cursor.execute(sql_insert_update_query, i)

In [198]:
update_db.dtypes

ID_Kraj       float64
Kraj           object
Region         object
Start_Data     object
End_Data       object
dtype: object